<a href="https://colab.research.google.com/github/amanbs/ECG_classification/blob/master/ECG_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Importing data into colab
from google.colab import files
uploaded=files.upload()

Saving Diagnostics.csv to Diagnostics.csv


In [ ]:
## Converting to df
import pandas as pd
data=pd.read_csv('Diagnostics.csv')


In [ ]:
feature_set=data[['VentricularRate','AtrialRate', 'QRSDuration', 'QTInterval', 'QTCorrected', 'RAxis','TAxis', 'QRSCount', 'QOnset', 'QOffset', 'TOffset']]

In [ ]:
#feature_set.head()

In [ ]:
#data.head()

In [ ]:
## Make output variable
# Import label encoder
from sklearn import preprocessing
import numpy as np


# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
data['Rhythm']= label_encoder.fit_transform(data['Rhythm'])


Z=np.asarray(data['Rhythm'])


In [ ]:
# Standard PyTorch imports
import torch


# Make device agnostic code
#device = "cuda" if torch.cuda.is_available() else "cpu"
#device

In [ ]:
import torch
##Turn data into tensors
feature = torch.tensor(feature_set.values).type(torch.float)

#feature = torch.from_numpy(feature_set).type(torch.float)
out = torch.from_numpy(Z).type(torch.LongTensor)
print(feature[:5], out[:5])

tensor([[117., 234., 114., 356., 496.,  81., -27.,  19., 208., 265., 386.],
        [ 52.,  52.,  92., 432., 401.,  76.,  42.,   8., 215., 261., 431.],
        [ 67.,  67.,  82., 382., 403.,  88.,  20.,  11., 224., 265., 415.],
        [ 53.,  53.,  96., 456., 427.,  34.,   3.,   9., 219., 267., 447.],
        [162., 162., 114., 252., 413.,  68., -40.,  26., 228., 285., 354.]]) tensor([1, 7, 5, 7, 0])


In [ ]:
#Split to train and split
from sklearn.model_selection import train_test_split
feature_train, feature_test, out_train, out_test = train_test_split(feature, out, test_size=0.2, random_state=42)


In [ ]:
NUM_FEATURES= 11
NUM_CLASSES= 11

In [ ]:
# Standard PyTorch imports
import torch
from torch import nn

# Make device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
##Setting up the model
from torch import nn
class ecgmodel(nn.Module):
  def __init__(self, input_features, output_features, hidden_units=128):
    super().__init__()
    self.linear_layer_stack=nn.Sequential(
        nn.Linear(in_features=input_features, out_features=hidden_units),
        nn.LeakyReLU(),
        nn.Linear(in_features=hidden_units, out_features=hidden_units),
        nn.LeakyReLU(),
        nn.Linear(in_features=hidden_units, out_features=output_features)

    )
    
  def forward (self, x):
    return self.linear_layer_stack(x)
model_1= ecgmodel(input_features=NUM_FEATURES,
                  output_features=NUM_CLASSES,
                  hidden_units=128).to(device)
print(model_1)

ecgmodel(
  (linear_layer_stack): Sequential(
    (0): Linear(in_features=11, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=128, out_features=11, bias=True)
  )
)


In [ ]:
## Setup a loss function  
loss_fn = nn.CrossEntropyLoss()


In [ ]:
#Calculate Accuracy
def accuracy_fn(y_true, y_pred):
  correct= torch.eq(y_true, y_pred).sum().item()
  acc=(correct/len(y_pred))*100
  return acc

In [ ]:
#Setup Optimizer
optimizer = torch.optim.Adam(params=model_1.parameters(),
                            lr=0.0001)

In [ ]:
## Training loop 

torch.manual_seed(42)

epochs = 10000

feature_train, out_train= feature_train.to(device), out_train.to(device)
feature_test, out_test= feature_test.to(device), out_test.to(device)

#Build training and evaluation loop
for epoch in range(epochs):
  model_1.train()

  # Forward pass
  y_logits=model_1(feature_train)
  y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)
  

  #Calculate loss
  loss= loss_fn(y_logits,
               out_train)
  #Accuracy not necesary
  acc= accuracy_fn(y_true= out_train,
                 y_pred=y_pred)
  #Optimizer
  optimizer.zero_grad()

  #Loss Backward
  loss.backward()

  #Optimizer Step (gradient descent)
  optimizer.step()

  #Testing
  model_1.eval()
  with torch.inference_mode():
      # 1. Forward pass
      test_logits = model_1(feature_test)
      test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
      # 2. Calculate test loss and accuracy
      test_loss = loss_fn(test_logits, out_test)
      test_acc = accuracy_fn(y_true=out_test,
                             y_pred=test_pred)

  # Print out what's happening
  if epoch % 1000 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%")


Epoch: 0 | Loss: 0.34854, Acc: 88.63% | Test Loss: 0.58745, Test Acc: 85.26%
Epoch: 1000 | Loss: 0.29132, Acc: 90.81% | Test Loss: 0.65918, Test Acc: 84.93%
Epoch: 2000 | Loss: 0.25579, Acc: 92.02% | Test Loss: 0.77788, Test Acc: 84.23%
Epoch: 3000 | Loss: 0.23330, Acc: 92.63% | Test Loss: 0.90431, Test Acc: 84.46%
Epoch: 4000 | Loss: 0.21548, Acc: 93.17% | Test Loss: 1.03087, Test Acc: 84.32%
Epoch: 5000 | Loss: 0.20251, Acc: 93.41% | Test Loss: 1.15187, Test Acc: 84.18%
Epoch: 6000 | Loss: 0.19235, Acc: 93.72% | Test Loss: 1.27525, Test Acc: 83.80%
Epoch: 7000 | Loss: 0.18352, Acc: 93.99% | Test Loss: 1.37677, Test Acc: 83.24%
Epoch: 8000 | Loss: 0.17587, Acc: 94.22% | Test Loss: 1.48656, Test Acc: 82.63%
Epoch: 9000 | Loss: 0.16926, Acc: 94.46% | Test Loss: 1.57996, Test Acc: 82.44%


In [ ]:
# Make predictions
model_1.eval()
with torch.inference_mode():
    y_logits = model_1(feature_test)

In [ ]:
# Turn predicted logits in prediction probabilities
y_pred_probs = torch.softmax(y_logits, dim=1)

# Turn prediction probabilities into prediction labels
y_preds = y_pred_probs.argmax(dim=1)

# Compare first 10 model preds and test labels
print(f"Predictions: {y_preds[:10]}\nLabels: {out_test[:10]}")
print(f"Test accuracy: {accuracy_fn(y_true=out_test, y_pred=y_preds)}%")


Predictions: tensor([ 7,  0, 10,  9,  9,  9,  7,  1,  7,  7])
Labels: tensor([ 7,  0, 10,  9,  9,  9,  7,  1,  7,  7])
Test accuracy: 82.58215962441314%


In [ ]:
!pip -q install torchmetrics

from torchmetrics import Accuracy

# Setup metric and make sure it's on the target device
torchmetrics_accuracy = Accuracy().to(device)

# Calculate accuracy
torchmetrics_accuracy(y_preds, out_test)

tensor(0.8258)